In [ ]:

# coding: utf-8

# In[1]:

# imports
import itertools
import os
import re
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn import metrics
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier,     RandomForestRegressor, RandomForestClassifier
from datetime import date


# start to calculate history rate for different attributes 
id_columns = ['teacher_acctid','schoolid']
location_columns = ['school_city','school_state','school_district','school_county','school_zip']
all_columns = id_columns + location_columns

requirements_columns = ['projectid','date_posted','is_exciting','at_least_1_teacher_referred_donor','fully_funded','at_least_1_green_donation',
                        'great_chat','one_or_more_required']
donation_columns = ['is_teacher_acct','donation_total']
require_only_columns = ['is_exciting','at_least_1_teacher_referred_donor','fully_funded','at_least_1_green_donation','great_chat','one_or_more_required']


def calculate_possibility(df, title, columns=require_only_columns):
    df_p = df 
    alpha = 10
    for var in require_only_columns:
        mean = df_p[var].sum()/df_p['count'].sum()
        #print(mean)
        df_p[var] = (df_p[var]+mean*alpha)/(df_p['count']+alpha)
        df_p[var][df_p['count']==1] = 0      
        df_p.columns = df_p.columns.str.replace(var,"".join(title+'_'+var+'_rate'))
    df_p.rename(columns={'count': "".join(title+'_count')}, inplace=True)
    return df_p
def calculate_possibility_by_time(df,title,start_date,end_date,columns= require_only_columns):
    df_p = df[(df['date_posted']<end_date)&(df['date_posted']>=start_date)]
    alpha = 10
    for var in require_only_columns:
        mean = df_p[var].sum()/df_p['count'].sum()
        #print(mean)
        df_p[var] = (df_p[var]+mean*alpha)/(df_p['count']+alpha)
        df_p[var][df_p['count']==1] = 0      
        df_p.columns = df_p.columns.str.replace(var,"".join(title+'_'+var))
    df_p.rename(columns={'count': "".join(title+'_count')}, inplace=True)
    return df_p

def calculate_sum(df,name):
    #df_tmp = df[df['date_posted']<"2014-01-01"]
    df_tmp = df 
    name_group = df_tmp.groupby(name,as_index=False)
    count = pd.DataFrame({'count':name_group.size()}).reset_index()
    prev = name_group.sum()
    prev = pd.merge(prev,count,on=name)
    df_tmp = pd.merge(df_tmp[['projectid',name]],prev,how='left',on=name)
    return df_tmp

def calculate_by_time(df,name,start_date,end_date):
    df_tmp = df[(df['date_posted']<end_date)&(df['date_posted']>=start_date)]
    name_group = df_tmp.groupby(name,as_index=False)
    count = pd.DataFrame({'count':name_group.size()}).reset_index()
    prev = name_group.sum()
    prev = pd.merge(prev,count,on=name)
    df_tmp = pd.merge(df_tmp[['projectid','date_posted',name]],prev,how='left',on=name)
    df_tmp = df_tmp.drop('date_posted',axis=1)
    return df_tmp

def get_full_df():
    test_df = df
    #data = df[(df['cat']=='train')|(df['cat']=='val')]
    for var in all_columns:
        #get columns of requirements 
        name_tmp = df[np.append(requirements_columns,var)]
        # calculate sum accroding to different attribute
        sum_tmp = calculate_sum(name_tmp,var)
        #calculate the possibility of different attribute
        pos_tmp = calculate_possibility(sum_tmp,var)
        # merge the result to the main df dataframe
        test_df = pd.merge(test_df,pos_tmp,how='left',on=['projectid',var])
        print("finish merge for "+ var)
    for var in donation_columns:
        name_tp = df[np.append(donation_columns,var)]
        sum_tmp = calculate_sum(name_tmp,var)
        pos_tmp = calculate_possibility(sum_tmp,var)
        test_df = pd.merge(test_df,pos_tmp,how='left',on=['projectid',var])
        print("finish merge for "+ var)
    return test_df


# this function is not finished yet
def get_date_partition(start_date, end_date, interval):
    intervals = []
    start_time = pd.to_datetime(start_date)
    end_time = pd.to_datetime(end_date)
    bet = end_time - start_time
    times = bet/interval
    for i in range(0,times):
        end = start_time + interval
        start_time.dt.strftime()



print('loading the data...')
projects = pd.read_csv('./data/projects.csv')
outcomes = pd.read_csv('./data/outcomes.csv')
print('complete..')


# In[3]:

print(outcomes.shape)
outcomes = outcomes.sort_values(by='projectid')
outcomes.fillna(method='pad')
outcomes.head()


# In[4]:

projects = projects.sort_values(by='projectid')
projects.fillna(method='pad')
projects.head()
df = pd.merge(projects, outcomes, how='left', on='projectid')
print(df.shape)
df[df['is_exciting'].isnull()].shape  # all the test is_exciting value is null 



# In[6]:

resources = pd.read_csv('./data/resources.csv')
print(resources.shape)
resources['cost'] = resources['item_unit_price']*resources['item_quantity']
#resources.head()
total_sum = resources[['projectid','cost','item_quantity']].groupby('projectid',as_index=False).aggregate(np.sum)
total_sum['avg_cost'] = total_sum['cost']/total_sum['item_quantity']
total_sum.head()
df = pd.merge(df, total_sum, how='left', on='projectid')

for var in resources['project_resource_type'].unique()[0:6]:
    tmp = resources[(resources['project_resource_type']==var)]
    tmp_sum= tmp[['projectid','cost']].groupby('projectid',as_index=False).aggregate(np.sum)
    tmp_sum.rename(columns={'cost': "".join(var+'_cost')}, inplace=True)
    df = pd.merge(df,tmp_sum,how='left',on ='projectid')
    df["".join(var+'_cost')].fillna(0)

print(df.shape)


df['price_per_student'] = df['cost']/df['students_reached']

essays = pd.read_csv('./data/essays.csv')
essays.fillna("",inplace=True)
essays['essay_length'] = essays['essay'].str.len()
essays['title_length'] = essays['title'].str.len()
df = pd.merge(df,essays[['projectid','essay_length','title_length']],on='projectid',how='left')


#add category for the data 
df['cat'] = "train"
df['cat'][df["date_posted"]<"2010-01-01"] = "nouse"
# valadation set
df['cat'][df["date_posted"]>="2013-01-01"] = "val"
# test set
df['cat'][df["date_posted"]>="2014-01-01"]= "test"
df = df[df['cat']!="nouse"]

df['is_exciting'][df['is_exciting']=="t"] = 1
df['is_exciting'][df['is_exciting']=="f"] = 0
df['is_exciting'].fillna(0,inplace=True)

df["at_least_1_teacher_referred_donor"][df["at_least_1_teacher_referred_donor"]=="t"] = 1
df["at_least_1_teacher_referred_donor"][df["at_least_1_teacher_referred_donor"]=="f"] = 0
df["at_least_1_teacher_referred_donor"].fillna(0,inplace=True)

df["great_chat"][df["great_chat"]=="t"] = 1
df["great_chat"][df["great_chat"]=="f"] = 0
df["great_chat"].fillna(0,inplace=True)

df["fully_funded"][df["fully_funded"]=="t"] = 1
df["fully_funded"][df["fully_funded"]=="f"] = 0
df["fully_funded"].fillna(0,inplace=True)

df["at_least_1_green_donation"][df["at_least_1_green_donation"]=="t"] = 1
df["at_least_1_green_donation"][df["at_least_1_green_donation"]=="f"] = 0
df["at_least_1_green_donation"].fillna(0,inplace=True)

df["donation_from_thoughtful_donor"][df["donation_from_thoughtful_donor"]=="t"] = 1
df["donation_from_thoughtful_donor"][df["donation_from_thoughtful_donor"]=="f"] = 0
df["donation_from_thoughtful_donor"].fillna(0,inplace=True)

df["three_or_more_non_teacher_referred_donors"][df["three_or_more_non_teacher_referred_donors"]=="t"] = 1
df["three_or_more_non_teacher_referred_donors"][df["three_or_more_non_teacher_referred_donors"]=="f"] = 0
df["three_or_more_non_teacher_referred_donors"].fillna(0,inplace=True)

df["one_non_teacher_referred_donor_giving_100_plus"][df["one_non_teacher_referred_donor_giving_100_plus"]=="t"] = 1
df["one_non_teacher_referred_donor_giving_100_plus"][df["one_non_teacher_referred_donor_giving_100_plus"]=="f"] = 0
df["one_non_teacher_referred_donor_giving_100_plus"].fillna(0,inplace=True)

df['teacher_referred_count'][df['teacher_referred_count']<1] = 0
df['teacher_referred_count'][df['teacher_referred_count']>=1] = 1
df['teacher_referred_count'].fillna(0,inplace=True)

df['non_teacher_referred_count'][df['non_teacher_referred_count']<1] = 0
df['non_teacher_referred_count'][df['non_teacher_referred_count']>=3] = 1
df['non_teacher_referred_count'].fillna(0,inplace=True)

one_or_more_required = ['three_or_more_non_teacher_referred_donors','one_non_teacher_referred_donor_giving_100_plus',
                       'donation_from_thoughtful_donor']
df['one_or_more_required'] = df[one_or_more_required].sum(axis=1)
df['one_or_more_required'][df['one_or_more_required']>=1] = 1
df['one_or_more_required'][df['one_or_more_required']<1] = 0
# this was indicated in the great_chat no need to cal again 
#df['great_messages_proportion'].fillna(0,inplace=True)
#df['great_messages_proportion'][df['great_messages_proportion'] >= 62] = True
#df['great_messages_proportion'][df['great_messages_proportion'] < 62] = False
#df['great_messages_proportion'].apply(lambda x: 1 if x else 0)


# add time tag columns for the data, maybe used for calculate history features
df["year"] = df["date_posted"].apply(lambda x: x.split("-")[0])
df["month"] = df["date_posted"].apply(lambda x: x.split("-")[1])
df["day"] = df["date_posted"].apply(lambda x: x.split("-")[2])
# convert time to int and sort by time
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)
df['day'] = df['day'].astype(int)



df['school_ncesid'] = df['school_ncesid'].apply(str)
df['school_zip'] = df['school_zip'].apply(str)

df['grade_level'][df['grade_level']=="Grades PreK-2"] = 0.0
df['grade_level'][df['grade_level']=="Grades 3-5"] = 1.0
df['grade_level'][df['grade_level']=="Grades 6-8"] = 2.0
df['grade_level'][df['grade_level']=="Grades 9-12"] = 3.0

df['poverty_level'][df['poverty_level']=='highest poverty'] = 3.0
df['poverty_level'][df['poverty_level']=='high poverty'] = 2.0
df['poverty_level'][df['poverty_level']=='moderate poverty'] = 1.0
df['poverty_level'][df['poverty_level']=='low poverty'] = 0.0

df.sort('date_posted',inplace=True)

"""
df['date_posted'] = pd.to_datetime(df['date_posted'])
ref_date = "2010-01-01"
ref_date = pd.to_datetime(ref_date)
print(ref_date)
df['daysbet'] = df['date_posted'] - ref_date
df['daysbet'] = df['daysbet'].dt.days
df['monthbet'] = df['date_posted'] - ref_date
df['monthbet'] = df['monthbet'].dt.days
df['monthbet'] = df['monthbet']/30
"""



# merge donation information
donations = pd.read_csv('./data/donations.csv')
donation_df = pd.merge(projects,donations,how='left',on='projectid')
donation_df = donation_df[['projectid','teacher_acctid','schoolid','is_teacher_acct','donation_total']]
donation_df["is_teacher_acct"][donation_df["is_teacher_acct"]=="t"] = 1
donation_df["is_teacher_acct"][donation_df["is_teacher_acct"]=="f"] = 0
donation_df["is_teacher_acct"].fillna(0,inplace=True)
donation_df["donation_total"].fillna(0,inplace=True)
#donation_df2=donation_df.groupby(by=["projectid","date_posted","teacher_acctid","schoolid","school_district","school_city","school_zip"])[["count","is_teacher_acct","donation_total"]].sum()
#donation_df2.reset_index(inplace=True)

donor_sum = calculate_sum(donation_df,'teacher_acctid')
donor_sum['tis_teacher_acct'] = donor_sum['is_teacher_acct']/donor_sum['count']
donor_sum['donation_total'] = donor_sum['donation_total']/donor_sum['count']
donor_sum.columns.str.replace('is_teacher_acct','teacher_acctid_is_teacher_acct')
donor_sum.columns.str.replace('donation_total','teacher_acctid_donation_total')
df = pd.merge(df,donor_sum,how='left',on=['projectid','teacher_acctid'])


donor_sum_2 = calculate_sum(donation_df,'schoolid')
donor_sum_2['is_teacher_acct'] = donor_sum_2['is_teacher_acct']/donor_sum_2['count']
donor_sum_2['donation_total'] = donor_sum_2['donation_total']/donor_sum_2['count']
donor_sum_2.columns.str.replace('is_teacher_acct','schoolid_is_teacher_acct')
donor_sum_2.columns.str.replace('donation_total','schoolid_donation_total')
df = pd.merge(df,donor_sum_2,how='left',on=['projectid','schoolid'])

df.fillna(method='pad',inplace=True)


example = get_full_df()
example.head()
#data = df[(df['cat']=='train')|(df['cat']=='val')]


# In[38]:

date_columns = ['day','month','year']
feature_columns = example.columns - example.columns[0:4] - requirements_columns-['cat'] - one_or_more_required - sub_primary_requirements - ['day']

example.fillna(method='pad',inplace=True)

example.to_csv('origin_data.csv',index=False)


sub_primary_requirements = ['great_messages_proportion','teacher_referred_count','non_teacher_referred_count'] 
projects_numeric_columns = ['school_latitude','school_longitude','fulfillment_labor_materials',
                           'total_price_excluding_optional_support',
                           'total_price_including_optional_support','grade_level','poverty_level','students_reached']
projects_id_columns =['projectid','teacher_acctid','schoolid','school_ncesid']
projects_categorial_columns = projects.columns - projects_numeric_columns - projects_id_columns - ['date_posted']
print(projects_categorial_columns)
# the way to encode the category may be wrong

for var in projects_categorial_columns:
    le = LabelEncoder()
    example[var] = le.fit_transform(example[var])
example.tail()



# after get full df, do feature selection 
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
train = example[(example['cat']=='train')|(df['cat']=='val')]
data = train[feature_columns]
X = data
names = data.columns.values
Y = train['is_exciting']
rfr = RandomForestRegressor()
print("start to fit the data")
rfr.fit(X,Y)
print("finish fitting")
print( "Features sorted by their score:")
feature_list = sorted(zip(map(lambda x: round(x, 4), rfr.feature_importances_), names), 
             reverse=True)
print(feature_list)


from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
dctr = DecisionTreeRegressor()
print("start to fit the data")
dctr.fit(X,Y)
print("finish fitting")
print( "Features sorted by their score:")
feature_list_2 = sorted(zip(map(lambda x: round(x, 4), dctr.feature_importances_), names), 
             reverse=True)
print(feature_list_2)


# after feature selection, select to drop some columns
update_feature_columns = []
for (v,n) in feature_list:
    update_feature_columns.append(n)


update_feature_columns_2 = []
for (v,n) in feature_list_2:
    update_feature_columns_2.append(n)


used_feature = list(set(update_feature_columns[0:50]).union(set(update_feature_columns_2[0:50])))


for_use_columns = ['projectid','is_exciting','cat'] + used_feature
example[for_use_columns].to_csv('experiment_data.csv',index=False)

"""

# In[48]:

# load data 
train = example[example['cat']=='train']
val = example[example['cat']=='val']
train_val = example[example['cat']!='test']
test = example[example['cat']=='test']

X_train = train[used_feature]
X_val = val[used_feature]
X_train_val = train_val[used_feature]
X_test = test[used_feature]

Y_train = train['is_exciting']
Y_val = val['is_exciting']
Y_train_val = train_val['is_exciting']


# In[ ]:




# In[ ]:

from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor

# Fit regression model
params = {'n_estimators': 300, 'max_depth': 7, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls','verbose':1,}
clf = GradientBoostingRegressor(**params)
print ("start training")
clf.fit(X_train_val, Y_train_val)
print("finish training")


# In[48]:

print("start to predict val")
predict = clf.predict(X_val)


# In[49]:

predict[predict>0.5] = 1
predict[predict<0.5] = 0


# In[50]:

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_val.values, predict)
mse


# In[51]:

from sklearn.metrics import accuracy_score
accuracy_score(Y_val.values,predict)


# In[53]:

test = example[example['cat']=='test']
X_test = test[used_feature]
X_test.shape


# In[54]:

print("start to predict test value")
test_predict = clf.predict(X_test)


# In[55]:

test_predict[test_predict>0.5] = 1
test_predict[test_predict<0.5] = 0


# In[56]:

test_predict[test_predict==1].shape


# In[57]:

test_predict[test_predict==0].shape


# In[58]:

sample = pd.read_csv('./data/sampleSubmission.csv')
sample['is_exciting'] = test_predict.astype(int)


# In[59]:

sample.head()
sample.to_csv('predictions.csv', index = False)
"""


loading the data...
complete..
(619326, 12)
(664098, 46)
(3667217, 9)
(664098, 55)


/Users/hang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:179: SettingWithCopyWarning: 
A value is trying to be set o